In [ ]:
!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset/")

Skipping, found downloaded files in "./face-expression-recognition-dataset" (use force=True to force download)


In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import MaxPooling2D
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
#set up variables
folder_path = "/content/face-expression-recognition-dataset/images/"
picture_size = 48
batch_size  = 128

#train test splot
datagen_train  = ImageDataGenerator()
datagen_val = ImageDataGenerator()
train_set = datagen_train.flow_from_directory(folder_path+"train",
                                              target_size = (picture_size,picture_size),
                                              color_mode = "grayscale",
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=True)


test_set = datagen_val.flow_from_directory(folder_path+"validation",
                                              target_size = (picture_size,picture_size),
                                              color_mode = "grayscale",
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=False)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [ ]:
no_of_classes = 7
dropout_rate = 0.25

model = Sequential()

#1st CNN layer
model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(dropout_rate))

#2nd CNN layer
model.add(Conv2D(128,(5,5),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (dropout_rate))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (dropout_rate))

#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dropout_rate))

model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dropout_rate))


# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dropout_rate))

model.add(Dense(no_of_classes, activation='softmax'))



opt = Adam(learning_rate = 0.001)

In [ ]:
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 48, 48, 64)        640       
                                                                 
 batch_normalization_37 (Ba  (None, 48, 48, 64)        256       
 tchNormalization)                                               
                                                                 
 activation_37 (Activation)  (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d_25 (MaxPooli  (None, 24, 24, 64)        0         
 ng2D)                                                           
                                                                 
 dropout_37 (Dropout)        (None, 24, 24, 64)        0         
                                                                 
 conv2d_26 (Conv2D)          (None, 24, 24, 128)      

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Updated the file extension to '.keras' from '.h5'
checkpoint = ModelCheckpoint("./model.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [early_stopping, checkpoint, reduce_learningrate]

epochs = 48

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])


In [ ]:
history = model.fit_generator(generator=train_set,
                                steps_per_epoch=train_set.n//train_set.batch_size,
                                epochs=epochs,
                                validation_data = test_set,
                                validation_steps = test_set.n//test_set.batch_size,
                                callbacks=callbacks_list
                                )

Epoch 1/48


<ipython-input-38-981791b505f2>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_set,


225/225 [==============================] - ETA: 0s - loss: 0.7978 - accuracy: 0.6990
Epoch 1: val_accuracy did not improve from 0.64787
225/225 [==============================] - 17s 76ms/step - loss: 0.7978 - accuracy: 0.6990 - val_loss: 0.9851 - val_accuracy: 0.6460 - lr: 1.6000e-06
Epoch 2/48
225/225 [==============================] - ETA: 0s - loss: 0.7913 - accuracy: 0.7052
Epoch 2: val_accuracy did not improve from 0.64787
225/225 [==============================] - 21s 95ms/step - loss: 0.7913 - accuracy: 0.7052 - val_loss: 0.9857 - val_accuracy: 0.6449 - lr: 1.6000e-06
Epoch 3/48
225/225 [==============================] - ETA: 0s - loss: 0.7908 - accuracy: 0.7038
Epoch 3: val_accuracy did not improve from 0.64787
225/225 [==============================] - 17s 76ms/step - loss: 0.7908 - accuracy: 0.7038 - val_loss: 0.9848 - val_accuracy: 0.6464 - lr: 1.6000e-06
Epoch 4/48
225/225 [==============================] - ETA: 0s - loss: 0.7899 - accuracy: 0.7046
Epoch 4: val_accuracy di

In [ ]:
#4 layers
#0.25: 0.64787
#0.375: 0.57628
#0.5: 0.53977

In [ ]:
model.load_weights('model.keras')
score = model.evaluate_generator(test_set)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

<ipython-input-41-324b40535c15>:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(test_set)


Test loss: 0.9833560585975647
Test accuracy: 0.6470421552658081
